In [12]:
from functools import cache
from collections import Counter
from math import comb
from heapq import heappush, heappop, heapify
from iteration_utilities import duplicates, unique_everseen
import threading
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from numba import njit


In [1]:
# @cache
# def collision(row_limit):
#     @cache
#     def comb_recurs(n, k):
#         if k == 0 or k == n:
#             return 1
#         else:
#             return comb_recurs(n-1, k-1) + comb_recurs(n-1, k)
#     sorted_list = sorted(comb_recurs(n, k) for n in range(10, row_limit) for k in range(2, n//2+1))
#     return [i for i, count in Counter(sorted_list).items() if count > 1]

@cache
def comb_recurs(n, k):
    if k == 0 or k == n:
        return 1
    else:
        return comb_recurs(n-1, k-1) + comb_recurs(n-1, k)

@cache
def collision(row_limit):
    return sorted(i for i, count in Counter(comb_recurs(n, k) for n in range(10, row_limit) for k in range(2, n//2+1)).items() if count > 1)

@cache
def collision2(row_limit):  # sorting inside duplicates gives same performance 
    return sorted(unique_everseen(duplicates(comb_recurs(n, k) for n in range(10, row_limit) for k in range(2, n//2+1))))

def search_pascal_multiples_fast1(row_limit):
    return sorted(i for i, count in Counter(comb(n, k) for n in range(10, row_limit) for k in range(2, n//2+1)).items() if count > 1)

row_limit = 250

%timeit collision(row_limit)
%timeit collision2(row_limit)
%timeit search_pascal_multiples_fast1(row_limit)

71.2 ns ± 11.1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
73.6 ns ± 4.65 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
51.9 ms ± 3.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
# threading implementation

@cache
def comb_recurs(n, k):
    if k == 0 or k == n:
        return 1
    else:
        return comb_recurs(n-1, k-1) + comb_recurs(n-1, k)

@cache
def nonThread(row_limit):
    return sorted(i for i, count in Counter(comb_recurs(n, k) for n in range(10, row_limit) for k in range(2, n//2+1)).items() if count > 1)

def threaded(row_limit):
    with concurrent.futures.ProcessPoolExecutor() as executor:
        f1 = executor.submit(nonThread, row_limit)
        print(f1.result())

print(threaded(250))


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.